In [1]:
import pandas as pd
import numpy as np

In [2]:
macro_1 = '2023-09-15 AFO BASE_Macro1.xlsb'
oh_avance = '2023-09-15 ON HAND AVANCE.xlsb'

In [3]:
# Carga de Datos
df1 = pd.read_excel(macro_1 , usecols = 'A:J',engine = 'pyxlsb', sheet_name = 'Hoja3', dtype = 'string')

# Transformamos las columnas de Totales de String a Float data type
for i in df1.columns[6:].values:
    df1[i] = df1[i].astype('float')

In [4]:
# On Hand Avance
df2 = pd.read_excel(oh_avance , sheet_name = 'BASE 1' , dtype = 'string',
                    usecols = ['OS', 'OP', 'Producto', 'Tipo almacén', 'Ubicación', 'Ctd.'])

# Ctd. a float
df2['Ctd.'] = list(map(lambda x: float(x),df2['Ctd.']))

# BASE 2
df2_base2 = pd.read_excel(oh_avance, sheet_name = 'BASE 2', dtype = 'string')
df2_base2 = df2_base2.loc[: , ['UBICACIÓN','OBSERVACION']]

In [5]:
# Crear Columnas OS+OP+MAT y TP.UBI

# OS+OP+MAT
df2['OS+OP+MAT'] = df2['OS'] + df2['OP'] + df2['Producto']

In [6]:
# Cruce ELIMINAR OS

# Eliminar Duplicados columna Ubicación Base 2
df2_base2.drop_duplicates(subset = 'UBICACIÓN', inplace = True)

# 10.3 Join (BUSCARV en excel)
df2.set_index('Ubicación', inplace = True)
df2_base2.set_index('UBICACIÓN', inplace = True)
df2 = df2.join(df2_base2, how = 'left')
df2 = df2.reset_index()
df2 = df2.rename({'index':'Ubicación'}, axis = 1)

# 11.1 Rellenar con NO ELIMINAR aquellas celdas NA del Join
df2['OBSERVACION'].fillna('NO ELIMINAR', inplace = True)

# 11.2 Si OBSERVACION = ElIMINAR OS, OS+OP+MAT = 0
df2.loc[df2['OBSERVACION'] == 'ELIMINAR OS', 'OS+OP+MAT'] = '0'

# 11.3 Eliminamos la columna Observación
df2.drop(columns = 'OBSERVACION', axis = 1, inplace = True)

In [7]:

# TP.UBI
df2['TP.UBI'] = pd.NA

# TP UBI 8034
df2.loc[df2['Tipo almacén'] == '8032', 'TP.UBI'] = '8034'
df2.loc[df2['Tipo almacén'] == '8034', 'TP.UBI'] = '8034'
df2.loc[df2['Ubicación'].str.contains('9020-CRC'), 'TP.UBI'] = '8034'
df2.loc[df2['Ubicación'].str.contains('9020-REGUL-CRC'), 'TP.UBI'] = '8034'
df2.loc[df2['Ubicación'].str.contains('9020-SUBKIT'), 'TP.UBI'] = '8034'

# TP.UBI OTROS
df2.loc[(pd.isna(df2['TP.UBI']) == True), 'TP.UBI'] = 'OTROS'

# Nos quedamos con las columnas OS+OP+MAT y TP.UBI
df2 = df2.drop(['OS', 'OP', 'Tipo almacén', 'Ubicación', 'Producto'], axis = 1)

In [8]:
# Tabla Dinámica
df2 = pd.pivot_table(data = df2, values = 'OS+OP+MAT', index = 'OS+OP+MAT', columns = 'TP.UBI', aggfunc = np.sum)

# Renombrar columnas
df2 = df2.rename({'8034': 'OH 8034', 'OTROS': 'OH Otros'}, axis = 1)

# Rellenar NaN con 0
df2.fillna(0, inplace = True)

In [9]:
# Cruce Macro 1 y O.H
df1 = df1.set_index('OS+OP+Mat')
df1 = df1.join(df2, how = 'left')
df1 = df1.reset_index()
df1 = df1.set_index('OS+OP')

In [10]:
# Columnas Adicionales y sus cálculos

# OH Total
df1['OH Total'] = df1['OH 8034'] + df1['OH Otros']
df1['OH Total'].fillna(0, inplace = True)

In [11]:
# Delta
df1['Delta'] = df1['OH Total'] - df1['TFaltanteBO']
df1['Delta'].fillna(0, inplace = True)

In [12]:
# Dejar vacío todo lo que tenga 0
df1.loc[df1['OH 8034'] == 0, 'OH 8034'] = pd.NA
df1.loc[df1['OH Otros'] == 0, 'OH Otros'] = pd.NA

In [13]:
df1.to_excel('Macro1xOnHand_v3.xlsx')